<a href="https://colab.research.google.com/github/Zennyth/Exam-Python/blob/main/Copie_de_itiiPythonExam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Devoir Python

Vous devez rendre votre devoir sur GitHub.
Vous avez le droit a tout vos documents et a internet

1. votre depot doit etre privé
2. vous devez inviter comme colaborateur votre chargé de TD/TP
3. Seul le dernier commit avant la fin de la séance sera corrigé.


Ex 1: Integrale de Romberg

Ecrire une fonction integ_romberg(f, a, b, epsilon=1e-6) permettant de calculer l’intégrale numérique de la fonction f entre les bornes a et b avec une précision epsilon selon la méthode de Romberg (https://fr.wikipedia.org/wiki/M%C3%A9thode_de_Romberg).

Il s’agit d’une méthode qui permet d’améliorer les méthodes usuelles de calcul numérique des intégrales, comme la méthode des Trapèzes.
On montre qu’en combinant judicieusement les valeurs obtenues par la méthode des Trapèzes pour différentes subdivisons de l’intervalle d’intégration, on augmente l’ordre de convergence (sa vitesse de convergence). 


In [71]:
#Mathis Figuet

def rombderg(f,a,b,e):
  """Romberg Integration :
  3 int ( f ( t ), t=a..b) avec n étapes """
  #On prépare un tableau qui va nous servir pour faire la somme
  r = []
  #On initie le premier h
  h=b-a
  s = 1
  #On calcul la première itération du calcul trouvé dans la section algorithme
  r.append([0.5*h*(f(a)+f(b))])
  while True : # On va boucler tant que la différence des deux termes succesifs n'est pas plus petite que epsilon => e
    #On update h en divisant par 2
    h=h/2
    #On initie la somme
    sum=0
    #On parcours T : T(2**n)
    for k in range(1,2**s,2):
      #On update la somme avec la formule f(a+k*h)
      sum = sum + f(a+(k*h))
      #On update notre valeur avec 1/2*n(n-1) + somme*h
      rowi = [0.5*r[s-1][0]+sum*h]
      #On parcours R
      for j in range(1, s+1):
        #On calcul la valeur grace a la formule trouvé dans la section redondance
        rij = rowi[j-1] + (rowi[j-1]-r[s-1][j-1])/(4**j-1)
        rowi.append(rij)
      #On ajoute le resultat dans le tableau
      r.append(rowi)
      if abs(r[s][s]-r[s][s-1]) <= e : # Tant que la valeur absolue n'est pas plus petite que epsilon on n'arrete pas le programme
        return r
      s = s+1 # On incrémente s chaque fois que la différence n'est pas assez petite pour calculer une nouvelle série d'approximations

def f(x):
  return 2/(1+x**2)

rombderg(f,-1,2,1e-6)

[[2.0999999999999996],
 [3.45, 3.9000000000000004],
 [3.136764705882353, 3.0323529411764705, 2.9745098039215683],
 [3.565512912482066, 3.708428981348637, 3.7535007173601147, 3.76586565249406],
 [2.322082298937662,
  1.9076054277561942,
  1.7875505241833647,
  1.7563449655615115,
  1.748464492279423],
 [2.438828530626999,
  2.477743941190111,
  2.515753175419039,
  2.527311947660875,
  2.5303353475906767,
  2.5310996397268557],
 [2.921980407533614,
  3.0830310331691524,
  3.1233835059677553,
  3.133028431849481,
  3.135403790611005,
  3.1359952553646906,
  3.136142971021745],
 [3.369564929677909,
  3.518759770392674,
  3.547808352874242,
  3.554545255206091,
  3.556198262356509,
  3.556609596150982,
  3.556712310275716,
  3.5567379813578897],
 [3.7062983672206444,
  3.8185428464015563,
  3.8385283848021485,
  3.8431429884835437,
  3.8442747443003182,
  3.844556343989413,
  3.8446266606555515,
  3.8446442346255436,
  3.8446486277933665],
 [3.944331268865742,
  4.023675569414108,
  4.0373

Ex 2: Équation d’état de l’eau à partir de la dynamique moléculaire

Afin de modéliser les planètes de type Jupiter, Saturne, ou même des exo-planètes très massives (dites « super-Jupiters »), la connaissance de l’équation d’état des composants est nécessaire. Ces équations d’état doivent être valables jusqu’à plusieurs centaines de méga-bar ; autrement dit, celles-ci ne sont en aucun cas accessibles expérimentalement. On peut cependant obtenir une équation d’état numériquement à partir d’une dynamique moléculaire.

Le principe est le suivant : on place dans une boite un certain nombre de particules régies par les équations microscopiques (Newton par exemple, ou même par des équations prenant en considération la mécanique quantique) puis on laisse celles-ci évoluer dans la boite ; on calcule à chaque pas de temps l’énergie interne à partir des intéractions électrostatiques et la pression à partir du tenseur des contraintes. On obtient en sortie l’évolution du système pour une densité fixée (par le choix de taille de la boite) et une température fixée (par un algorithme de thermostat que nous ne détaillerons pas ici).

On se propose d’analyser quelques fichiers de sortie de tels calculs pour l’équation d’état de l’eau à très haute pression. Les fichiers de sortie sont disponibles ici; leur nom indique les conditions thermodynamiques correspondant au fichier, p.ex. 6000K_30gcc.out pour T=6000
K et ρ=30 gcc. Le but est, pour chaque condition température-densité, d’extraire l’évolution de l’énergie et de la pression au cours du temps, puis d’en extraire la valeur moyenne ainsi que les fluctuations. Il arrive souvent que l’état initial choisi pour le système ne corresponde pas à son état d’équilibre, et qu’il faille donc « jeter » les quelques pas de temps en début de simulation qui correspondent à cette relaxation du système. Pour savoir combien de temps prend cette relaxation, il sera utile de tracer l’évolution au cours du temps de la pression et l’énergie pour quelques simulations. Une fois l’équation d’état P(ρ,T) et E(ρ,T) extraite, on pourra tracer le réseau d’isothermes.

Ex 3: Le problème du voyageur de commerce

Le problème du voyageur de commerce est un problème d’optimisation consistant à déterminer le plus court chemin reliant un ensemble de destinations. Il n’existe pas d’algorithme donnant la solution optimale en un temps raisonnable (problème NP-complet), mais l’on peut chercher à déterminer des solutions approchées. On va se placer ici dans le cas d’un livreur devant desservir une seule fois chacune desndestinations d’une ville américaine où les rues sont agencées en réseau carré. 

On utilise la « distance deManhattan »  entre deux points 𝐴(𝑥𝐴,𝑦𝐴) et 𝐵(𝑥𝐵,𝑦𝐵) : 𝑑(𝐴,𝐵) =|𝑥𝐵−𝑥𝐴|+|𝑦𝐵−𝑦𝐴|.

En outre, on se place dans le cas où les coordonnées des destinations sont entières, comprises entre 0 (inclus) et TAILLE = 50 (exclus). Deux destinations peuvent éventuellement avoir les mêmes coordonnées. Les instructions suivantes doivent permettre de définir les classes nécessaires (Ville et Trajet) et de développer un algorithme approché (heuristiques) : l’algorithme du plus proche voisin. 

Seules la librairie standard et la librairie numpy sont utilisables si nécessaire. Implementer les classes et methodes suivante:


Classe Ville:
* __init__(): initialisation d’une ville sans destination.
* aleatoire(n): création de n destinations aléatoires.
* nb_trajet(): retourne le nombre total (entier) de trajets :(𝑛−1)!/2(utilisermath.factorial()).
* distance(i, j): retourne la distance (Manhattan) entre les deux destinations de numéro i et j


Classe Trajet:
* __init__(ville, etapes=None): initialisation sur une ville. Si la liste etapes n’est pas spécifiée, le trajet par défaut est celui suivant les destinations de ville.
* longueur(): retourne la longueur totale du trajetbouclé(i.e. revenant à son point de départ).


Plus proche voisin:
* Ville.plus_proche(i, exclus=[]): retourne la destination la plus proche de la destinationi(au sens de Ville.distance()), hors les destinations de la liste exclus
* Ville.trajet_voisins(depart=0): retourne un Trajet déterminé selon l’heuristique des plus proches voisins (i.e. l’étape suivante est la destination la plus proche hors les destinations déjà visitées) en partant de l’étape initiale depart

Optimisation:
* Proposer un algorithme qui propose une meilleur alternative au "plus proche voisin".

Interface:
* Ville.figure(trajet=None): Afficher le plande la ville et le trajet obtenue (utiliser matplotlib.step()pour des trajets de type « Manhattan »)

In [68]:
import random
import math

class trajet:
  def __init__(self, ville, etapes=None):
    self.ville = ville
    if etapes != None:
      self.etapes = etapes
    else:
      self.etapes = ville.destinations
  
  def longueur():
    sum = 0
    for i in range(len(self.etapes)-2):
      sum += self.ville.distance(i, i+1)
    return sum

class Ville:
   def __init__(self, nom):
     # On init les destinations avec un tableau vide
      self.nom = nom
      self.destinations = []
      self.aleatoire(10)
    
   def aleatoire(self, n):
     # On parcours n en ajoutant un tableau de deux random entre 0-49 dans destinations
     for i in range(n-1):
       self.destinations.append([random.randint(0, 49),random.randint(0, 49)])
   
   def nb_trajet(self):
     # On calcul n-1!/2 ou n est la length du tableau destinations
     return math.factorial(len(self.destinations)-1)/2

   def distance(self, i, j):
     distanceX = self.destinations[i][0] - self.destinations[j][0] # On selectionne les x donc on prend 0 en index
     distanceY = self.destinations[i][1] - self.destinations[j][0] # On selectionne les y donc on prend 1 en index
     distance = math.sqrt( (distanceX**2) + (distanceY**2) ) #Distance de manhattan
     return distance
   
   def plus_proche(self, i, exclus=[]):
     proche = [None, 50] #Initilasation de proche 0: inex, 1: distance
     for j in range(len(self.destinations)-1): #parcours de destination
       if j != i and self.destinations.count(i) == 0: #Ne pas prendre i et les exclus
         actuel = self.distance(i,j) #distance
         if actuel < proche[1]: #Choisir minimu des deux distances
           proche = [j, actuel]
     return proche

   def trajet_voisins(self, depart=0):
     etapes = [] #Initialisation de etapes
     exclus = [] #Initialisation de exclus
     for i in range(depart, len(self.destinations)-1): #parcours de destination
       plusProche = self.plus_proche(i, exclus) #Prends plus proche
       exclus.append(plusProche[0]) #Ajoute l'index dans exclus
       etapes.append(self.destinations[i]) #Ajoute les coordonnées dans etapes

     return trajet(self,etapes=etapes)

newVille = Ville("monaco")
print(newVille.destinations)
print(newVille.nb_trajet())
print(newVille.distance(1,2))
print(newVille.plus_proche(1))
print(newVille.trajet_voisins().etapes)

[[26, 35], [31, 40], [45, 5], [18, 17], [19, 45], [40, 43], [40, 38], [28, 8], [29, 43]]
20160.0
14.866068747318506
[5, 9.0]
[[26, 35], [31, 40], [45, 5], [18, 17], [19, 45], [40, 43], [40, 38], [28, 8]]
